tb - 10/18/2021 - Reproduce results from Ryan and output (1) number of free parameters ; and (2) mean error statistics of the net shortwave flux prediction (in $W m^{-2} $) from best-performing model.

Kind instruction email from Ryan L below:

•	The Python library, called ml4rt, is available here.  It's under the MIT licence, so feel free to use/modify for any non-commercial purposes.
•	To read the files with learning examples (all NetCDF files in the directories tropical_sites and non_tropical_sites), use example_io.read_file, which you can find here.  I recommend always using the options exclude_summit_greenland = True (the RRTM, which is the model we're trying to emulate, made some errors on these profiles, so the labels are inaccurate) and max_heating_rate_k_day = numpy.inf (in other words, don't filter by heating rate).  example_io.read_file returns a dictionary, and the format of this dictionary is documented in the method string for example_io.read_file.
•	One file (non_tropical_sites/learning_examples_20170101-20181224.nc) contains normalization parameters, used to normalize data before input to the neural net.
•	To normalize data, use the method normalization.normalize_data (here), where new_example_dict contains the examples you want to normalize and training_example_dict contains normalization parameters (read from non_tropical_sites/learning_examples_20170101-20181224.nc).  For some example usage of normalization.normalize_data, see neural_net.read_file_for_generator (here), where the method is called 3 times.
•	To read the model (a U-net++), use the method neural_net.read_model (here).  Make sure the metafile (model_metadata.dill) is always in the same directory as the actual model (model.h5).
•	To read the data and prepare it for input to the the neural net (i.e., pre-process it the same way the training data was pre-processed), use neural_net.create_data (here) or neural_net.data_generator (here).
•	To apply the trained neural net to some data, use neural_net.create_data or neural_net.data_generator to read/pre-process the data, then use neural_net.apply_model (here) with net_type_string = "u_net".  I recommend verbose = True and num_examples_per_batch = 5000 (but you may need it smaller if you have a computer with limited RAM).  predictor_matrix will be the predictor_matrix returned by neural_net.create_data or neural_net.data_generator.

I hope this is enough to get you started.  Please let me know if you have any questions about the code.  It's a bit complicated because [a] the neural net has both scalar and vector inputs (the scalars being solar zenith angle and surface albedo, the vectors being 1-D profiles of other variables); [b] the neural net has both scalar and vector outputs (the scalars being TOA upwelling flux, sfc downwelling flux, and net flux; the vector being a 1-D profile of heating rates; other possible vector outputs are full profiles of upwelling and downwelling flux, but I've since stopped predicting these, since NWP models don't require them from a paramzn).

# Imports

In [1]:
from ml4rt.io.example_io import *
from ml4rt.utils.normalization import *
from ml4rt.machine_learning.neural_net import *
from ml4rt.machine_learning.u_net_architecture import *
from ml4rt.machine_learning.u_net_pp_architecture import *

import dill

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [2]:
fz = 15
lw = 4
siz = 100

plt.rc('text', usetex=False)
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'STIXGeneral'
plt.rc('font', family='serif', size=fz)
mpl.rcParams['lines.linewidth'] = lw

# Read Files

In [3]:
read_file

<function ml4rt.io.example_io.read_file(netcdf_file_name, exclude_summit_greenland=False, max_heating_rate_k_day=41.5, id_strings_to_read=None, allow_missing_ids=False)>

In [4]:
path_data = '/work/FAC/FGSE/IDYST/tbeucler/default/tbeucler/2021_Ryan_Lagerquist_SW/'

In [5]:
read2020 = read_file(path_data+'tropical_sites/learning_examples_2020.nc',
                     exclude_summit_greenland=True)

/work/FAC/FGSE/IDYST/tbeucler/default/tbeucler/ml4rt/notebooks_tbeucler/ml4rt/io/example_io.py:304: UserWarning: Removing 0 of 264638 examples (profiles), because they are at Summit GL.
  warnings.warn(warning_string)


In [6]:
read2020.keys()

dict_keys(['example_id_strings', 'scalar_predictor_names', 'vector_predictor_names', 'scalar_target_names', 'vector_target_names', 'scalar_predictor_matrix', 'vector_predictor_matrix', 'scalar_target_matrix', 'vector_target_matrix', 'valid_times_unix_sec', 'standard_atmo_flags', 'heights_m_agl'])

In [7]:
read2020['scalar_predictor_matrix'].shape

(264638, 6)

In [8]:
read2020['scalar_predictor_names']

['zenith_angle_radians',
 'albedo',
 'latitude_deg_n',
 'longitude_deg_e',
 'column_liquid_water_path_kg_m02',
 'column_ice_water_path_kg_m02']

In [9]:
read2020['vector_predictor_matrix'].shape

(264638, 73, 12)

In [10]:
read2020['vector_predictor_names']

['pressure_pascals',
 'temperature_kelvins',
 'specific_humidity_kg_kg01',
 'liquid_water_content_kg_m03',
 'ice_water_content_kg_m03',
 'liquid_water_path_kg_m02',
 'ice_water_path_kg_m02',
 'vapour_path_kg_m02',
 'upward_liquid_water_path_kg_m02',
 'upward_ice_water_path_kg_m02',
 'upward_vapour_path_kg_m02',
 'relative_humidity_unitless']

In [11]:
read2020['scalar_target_names']

['shortwave_surface_down_flux_w_m02', 'shortwave_toa_up_flux_w_m02']

In [12]:
read2020['scalar_target_matrix'].shape

(264638, 2)

In [13]:
read2020['vector_target_names']

['shortwave_down_flux_w_m02',
 'shortwave_up_flux_w_m02',
 'shortwave_heating_rate_k_day01',
 'shortwave_down_flux_increment_w_m03',
 'shortwave_up_flux_increment_w_m03']

In [14]:
read2020['vector_target_matrix'].shape

(264638, 73, 5)

In [45]:
read2018_nt = read_file(path_data+'non_tropical_sites/learning_examples_2018.nc',
                     exclude_summit_greenland=True)

In [46]:
read2018_nt['vector_predictor_matrix'].shape

(897508, 73, 12)

# Normalize data

## Load normalization parameters from non tropical sites

In [17]:
path_norm_param = path_data+'non_tropical_sites/learning_examples_20170101-20181224.nc'

In [18]:
norm_param = read_file(path_norm_param,
                     exclude_summit_greenland=True,
                     max_heating_rate_k_day)

/work/FAC/FGSE/IDYST/tbeucler/default/tbeucler/ml4rt/notebooks_tbeucler/ml4rt/io/example_io.py:304: UserWarning: Removing 0 of 100000 examples (profiles), because they are at Summit GL.
  warnings.warn(warning_string)


In [19]:
norm_param.keys()

dict_keys(['example_id_strings', 'scalar_predictor_names', 'vector_predictor_names', 'scalar_target_names', 'vector_target_names', 'scalar_predictor_matrix', 'vector_predictor_matrix', 'scalar_target_matrix', 'vector_target_matrix', 'valid_times_unix_sec', 'standard_atmo_flags', 'heights_m_agl'])

## Apply normalization 
Following [https://github.com/thunderhoser/ml4rt/blob/8ad36b52ead1c4870bdccaa804dee9a52144cce0/ml4rt/machine_learning/neural_net.py#L213]

In [20]:
normalize_data

<function ml4rt.utils.normalization.normalize_data(new_example_dict, training_example_dict, normalization_type_string, min_normalized_value=-1.0, max_normalized_value=1.0, separate_heights=False, apply_to_predictors=True, apply_to_vector_targets=True, apply_to_scalar_targets=True)>

The normalization is automatically applied as part of data_generator (see _read_file_for_generator)

In [21]:
data_generator

<function ml4rt.machine_learning.neural_net.data_generator(option_dict, for_inference, net_type_string)>

# Read UNet++ trained by Ryan

In [22]:
read_model

<function ml4rt.machine_learning.neural_net.read_model(hdf5_file_name)>

In [23]:
path_model = path_data + 'actual_model/model.h5'

In [24]:
Unetpp = read_model(path_model)

/work/FAC/FGSE/IDYST/tbeucler/default/tbeucler/miniconda3/envs/ml4rt/lib/python3.7/site-packages/tensorflow/python/keras/layers/core.py:1030: UserWarning: u_net_architecture is not loaded, but a Lambda layer uses it. It may cause errors.
  , UserWarning)
2021-10-20 18:26:34.540398: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2021-10-20 18:26:34.652630: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2894550000 Hz
2021-10-20 18:26:34.655155: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x565290e8ffd0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-10-20 18:26:34.655209: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-10-20 18:26:34.657771: I tensorflow/core/common_runtime/process_util.cc:147] Creating new thread pool with def

In [25]:
Unetpp.summary()

Model: "model_64"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_65 (InputLayer)           [(None, 73, 14)]     0                                            
__________________________________________________________________________________________________
block0-0_conv0 (Conv1D)         (None, 73, 64)       2752        input_65[0][0]                   
__________________________________________________________________________________________________
block0-0_conv0_activation (Leak (None, 73, 64)       0           block0-0_conv0[0][0]             
__________________________________________________________________________________________________
block0-0_conv0_bn (BatchNormali (None, 73, 64)       256         block0-0_conv0_activation[0][0]  
___________________________________________________________________________________________

In [26]:
Unetpp_total_p = 14952344
Unetpp_train_p = 14937052

# Test UNet++ trained by Ryan

## On shortwave heating rates and top/bottom fluxes

In [27]:
data_generator

<function ml4rt.machine_learning.neural_net.data_generator(option_dict, for_inference, net_type_string)>

### Loading model's metadata using dill

In [28]:
model_arc = dill.load(open(path_data+'actual_model/model_metadata.dill','rb'))

In [29]:
model_arc.keys()

dict_keys(['num_epochs', 'num_training_batches_per_epoch', 'training_option_dict', 'num_validation_batches_per_epoch', 'validation_option_dict', 'net_type_string', 'loss_function_or_dict', 'do_early_stopping', 'plateau_lr_multiplier'])

In [30]:
model_arc['net_type_string']

'u_net'

In [31]:
model_arc['validation_option_dict']

{'scalar_predictor_names': ['zenith_angle_radians', 'albedo'],
 'vector_predictor_names': ['pressure_pascals',
  'temperature_kelvins',
  'specific_humidity_kg_kg01',
  'liquid_water_content_kg_m03',
  'ice_water_content_kg_m03',
  'relative_humidity_unitless',
  'liquid_water_path_kg_m02',
  'ice_water_path_kg_m02',
  'vapour_path_kg_m02',
  'upward_liquid_water_path_kg_m02',
  'upward_ice_water_path_kg_m02',
  'upward_vapour_path_kg_m02'],
 'scalar_target_names': ['shortwave_surface_down_flux_w_m02',
  'shortwave_toa_up_flux_w_m02'],
 'vector_target_names': ['shortwave_heating_rate_k_day01'],
 'heights_m_agl': array([1.00e+01, 2.00e+01, 4.00e+01, 6.00e+01, 8.00e+01, 1.00e+02,
        1.20e+02, 1.40e+02, 1.60e+02, 1.80e+02, 2.00e+02, 2.25e+02,
        2.50e+02, 2.75e+02, 3.00e+02, 3.50e+02, 4.00e+02, 4.50e+02,
        5.00e+02, 6.00e+02, 7.00e+02, 8.00e+02, 9.00e+02, 1.00e+03,
        1.10e+03, 1.20e+03, 1.30e+03, 1.40e+03, 1.50e+03, 1.60e+03,
        1.70e+03, 1.80e+03, 1.90e+03, 2.0

In [32]:
model_arc['validation_option_dict']['example_dir_name'] = \
'/work/FAC/FGSE/IDYST/tbeucler/default/tbeucler/2021_Ryan_Lagerquist_SW/non_tropical_sites'

In [33]:
model_arc['validation_option_dict']['normalization_file_name'] = model_arc['validation_option_dict']['example_dir_name'] + \
'/learning_examples_20170101-20181224.nc'

In [34]:
model_arc['validation_option_dict']['first_time_unix_sec'] = 1514764800 # Jan 1, 2018 at 12:00:00AM
model_arc['validation_option_dict']['last_time_unix_sec'] = 1546300799 # Dec 31, 2018 at 11:59:59PM

In [35]:
Unetpp_gen = data_generator(model_arc['validation_option_dict'],True,model_arc['net_type_string'])

In [48]:
Unetpp_data = create_data(model_arc['validation_option_dict'],True, 
                          model_arc['net_type_string'],exclude_summit_greenland=True)

Reading training examples (for normalization) from: "/work/FAC/FGSE/IDYST/tbeucler/default/tbeucler/2021_Ryan_Lagerquist_SW/non_tropical_sites/learning_examples_20170101-20181224.nc"...

Reading data from: "/work/FAC/FGSE/IDYST/tbeucler/default/tbeucler/2021_Ryan_Lagerquist_SW/non_tropical_sites/learning_examples_2018.nc"...
Applying Z_SCORE normalization to predictors...
Applying MINMAX normalization to scalar targets...


In [37]:
Unetpp_data[0].shape

(897508, 73, 14)

In [38]:
Unetpp_data[1][0].shape

(897508, 73, 1)

In [39]:
Unetpp_data[1][1].shape

(897508, 2)

In [40]:
apply_model

<function ml4rt.machine_learning.neural_net.apply_model(model_object, predictor_matrix, num_examples_per_batch, net_type_string, verbose=False)>

In [41]:
Unetpp_apply = apply_model(Unetpp, Unetpp_data[0][:1000,:,:],1000,model_arc['net_type_string'])

In [42]:
Unetpp_apply[1].shape

(1000, 2)

In [43]:
Unetpp_apply[0].shape

(1000, 73, 1)

In [44]:
Unet_predictors = denormalize_data(Unetpp_data,model_arc['validation_option_dict'],'z_score')

TypeError: tuple indices must be integers or slices, not str

In [ ]:
plt.plot(np.mean(Unetpp_data[0][:1000,:,5],axis=0))